<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/use_image_as_input/training_spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ラベルの読み込み
import pickle
with open("/content/drive/MyDrive/pickle/label_five_class.pickle", mode="rb") as f:
   label = pickle.load(f)
f.close()

In [8]:
# データセット内訳チェック
import numpy as np

print(np.array(np.where(label == 0)).shape)
print(np.array(np.where(label == 1)).shape)
print(np.array(np.where(label == 2)).shape)
print(np.array(np.where(label == 3)).shape)
print(np.array(np.where(label == 4)).shape)

# NaNチェック
print(np.isnan(label).sum())

(1, 7755)
(1, 12748)
(1, 19873)
(1, 21386)
(1, 13970)
0


In [5]:
label.shape

(75732,)

In [9]:
import tensorflow as tf
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, InputLayer, Dense, Dropout, Activation, Flatten, concatenate, Conv1D, MaxPooling1D, Input, Reshape, Bidirectional, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
import time
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
from PIL import Image
import numpy as np

# NAME = "HC-vs-C-LSTM_2class"
# NAME = "HC-vs-A-LSTM_2class"
# NAME = "HC-vs-K-LSTM_2class"
# NAME = "HC-vs-H-LSTM_2class"
# NAME = "HC-vs-GDs-LSTM_2class"
# NAME = "H-vs-K-vs-A-vs-C-LSTM_4class"
#NAME = "HC-vs-H-vs-K-vs-A-vs-C-LSTM_5class"
# NAME = "H-vs-K-vs-A-vs-C-CNN_4class"
NAME = "HC-vs-Patients-CNN_5class"
# NAME = "HC-vs-GDs-CNN_2class"
# NAME = "HC-vs-H-CNN_2class"
# NAME = "HC-vs-K-CNN_2class"
# NAME = "HC-vs-A-CNN_2class"
# NAME = "HC-vs-C-CNN_2class"

#-----------------------------------初期設定-------------------------------------------
# tensorboardのログ
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# 学習打ち切り目安
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=10,
                )

# checkpointの設定
checkpoint = ModelCheckpoint(
                    filepath="saved_model/{}".format(NAME),
                    monitor='val_loss',
                    save_best_only=True,
                    period=1,
                )

# weightの初期化
# initializer = tf.initializers.he_normal()

#-----------------------------------学習モデル作成-------------------------------------------


#-----------------------------------Optimizer-------------------------------------------
# Optimizer: 多クラス分類+Adam
# model.compile(loss='sparse_categorical_crossentropy', 
#               optimizer= optimizers.Adam(lr=0.002, beta_1= 0.9, beta_2= 0.999),
#               metrics=['accuracy'],
#               )

# Optimizer: 多クラス分類+SGD
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer= optimizers.SGD(lr=5e-2, decay=1e-2),
#               metrics=['accuracy'],
#               )

# Optimizer: 2クラス分類
# model.compile(loss='binary_crossentropy',
#               optimizer= optimizers.SGD(lr=5e-3),
#               metrics=['accuracy'],
#               )

#-----------------------------------summer出力-------------------------------------------
# model.summary()

#-----------------------------------学習-------------------------------------------
# model.fit(X, train_label,
#           batch_size=5,
#           epochs=1000,
#           validation_split=0.2,
#           callbacks=[tensorboard])

# training parameters = kernel_size * (1Dだったらここは1, 2Dだったらここはkernel_size) * output_channels + num_bias
# 216 = 8 * 1 * 24 + 24

In [10]:
# 学習モデル構成
def lenet(INPUT_SHAPE, num_classes):
    """
    http://tecmemo.wpblog.jp/wp-content/uploads/2017/03/dl_lenet-01.png この表を参考に一部活性化関数を変更してLenetを定義
    """
    initializer = tf.initializers.he_normal()
    model = Sequential()
    # フィルターを6枚用意, 小窓のサイズ5×5, paddingによって入力と出力の画像サイズは同じ
    # from IPython.core.debugger import Pdb; Pdb().set_trace()
    model.add(Conv2D(
        64, kernel_size=3, padding="same",
        input_shape=INPUT_SHAPE, activation="relu",
        kernel_initializer=initializer
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(GlobalAveragePooling2D())

    # Flatten()はマトリックスを1次元ベクトルに変換する層
    # FCにつなぐために必要
    model.add(Dense(num_classes, kernel_initializer=initializer))
    model.add(Dropout(0.6))
    model.add(Activation("softmax"))
    model.summary()
    return model

In [11]:
# 入力データの加工

def conv_binary(number):
  V_left_filename = '/content/spectrogram_data/V_left/{}.png'.format(number)
  V_right_filename = '/content/spectrogram_data/V_right/{}.png'.format(number)
  V_left_img = Image.open(V_left_filename)
  V_right_img = Image.open(V_right_filename)
  V_left_img = V_left_img.resize((256, 512))
  V_right_img = V_right_img.resize((256, 512))
  # 色の変換も簡単ですが、できる色が制限されます。
  # print(img.mode) # RGBA
  V_left_rgb = np.array(V_left_img.convert('RGB'))
  V_right_rgb = np.array(V_right_img.convert('RGB'))
  # numpy配列の取得
  img_array = np.concatenate([V_left_rgb, V_right_rgb], axis=1)
  return img_array

In [17]:
# acquire the .csv name
TRAINS = range(len(label))
y = label
# split test and train
X_train, X_test, Y_train, Y_test = train_test_split(
    TRAINS, y, test_size=0.1, random_state=42)

In [21]:
def get_batch(batch_size, X, Y):
    """
    batchを取得する関数
    """
    SIZE = len(X)
    # n_batchs
    n_batchs = SIZE//batch_size
    # for でyield
    i = 0
    while (i < n_batchs):
        # print("doing", i, "/", n_batchs)
        # Y_batch = Y[(i * n_batchs):(i * n_batchs + batch_size)]
        Y_batch = Y[i:(i + batch_size)]
        
        #あるbatchのfilenameの配列を持っておく
        X_batch_name = X[i:(i + batch_size)]

        # filenameにしたがってバッチのtensorを構築
        X_batch = np.array([conv_binary(file) for file in X_batch_name])
        # from IPython.core.debugger import Pdb; Pdb().set_trace()
        # これで(batch_size, 28, 28, 1)のtrainのテンソルが作られる
        i += 1
        yield X_batch, Y_batch

In [30]:
model = lenet((512, 512, 3), 5)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.002, beta_1= 0.9, beta_2= 0.999),
              metrics=['accuracy'])
N_EPOCHS = 1000
BATCH_SIZE = 64
# for epoch in range(N_EPOCHS):
#     print("=" * 50)
#     print("epoch", epoch, "/", N_EPOCHS)
#     acc = []
    
#     # batch_size=1000でHDDからバッチを取得する
#     for X_batch, Y_batch in get_batch(BATCH_SIZE):
#         # from IPython.core.debugger import Pdb; Pdb().set_trace()
#         model.train_on_batch(X_batch, Y_batch)
#         score = model.evaluate(X_batch, Y_batch)
#         print("batch accuracy:", score[1])
#         acc.append(score[1])
#     print("Train accuracy", np.mean(acc))
#     # from IPython.core.debugger import Pdb; Pdb().set_trace()
#     score = model.evaluate(np.array(X_test), np.array(Y_test))
#     print("Test loss:", score[0])
#     print("Test accuracy:", score[1])

training_generator = get_batch(BATCH_SIZE, X_train, Y_train)
validation_generator = get_batch(BATCH_SIZE, X_test, Y_test)
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    steps_per_epoch=len(Y_train)//BATCH_SIZE,
    validation_steps=len(Y_test)//BATCH_SIZE,
    epochs=N_EPOCHS,
    shuffle=True,
    callbacks=[tensorboard]
)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_88 (Conv2D)           (None, 512, 512, 64)      1792      
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 64, 64, 128)     

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
1064/1064 [==============================] - 1132s 1s/step - loss: 139.4950 - accuracy: 0.2953 - val_loss: 2.3524 - val_accuracy: 0.2926
Epoch 2/1000
1064/1064 [==============================] - 0s 38us/step - loss: 22.3887 - accuracy: 0.3856


In [18]:
np.array(X_test).shape

(7574,)

In [19]:
np.array(Y_test).shape

(7574,)

In [ ]:
# データの読み込み 
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_right.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_right_69826.zip

In [ ]:
# データの読み込み
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_left_58106.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_right.zip

In [ ]:
# データの読み込み
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_left_48327.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_right.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_right_34533.zip